# Gene essentiality

In [1]:
from __future__ import print_function, division, absolute_import

import sys

import qminospy
from qminospy.me2 import ME_NLP

# python imports
from copy import copy
import re
from os.path import join
from collections import defaultdict
import pickle

# third party imports
import pandas
import cobra
from tqdm import tqdm
import numpy as np
import scipy

# COBRAme
import cobrame
from cobrame.util import building, mu, me_model_interface
from cobrame.io.json import save_json_me_model, save_reduced_json_me_model

# ECOLIme
import ecolime
from ecolime import (transcription, translation, flat_files, generics, formulas, compartments)
from ecolime.util.helper_functions import *

%load_ext autoreload
%autoreload 2
print(cobra.__file__)
print(cobrame.__file__)
print(ecolime.__file__)

/home/jt/Downloads/cobrapy-0.5.11/cobra/io/sbml3.py:24: UserWarning: Install lxml for faster SBML I/O
  warn("Install lxml for faster SBML I/O")
/home/jt/Downloads/cobrapy-0.5.11/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


/home/jt/Downloads/cobrapy-0.5.11/cobra/__init__.py
/home/jt/UCSD/cobrame-master/cobrame/__init__.py
/home/jt/UCSD/bacillusme-master/ecolime/__init__.py


In [2]:
eco_directory = join(flat_files.ecoli_files_dir, 'iJO1366.json')
ijo_directory = join(flat_files.ecoli_files_dir, 'iYO844.json')
uni_directory = join(flat_files.ecoli_files_dir, 'universal_model.json')

eco = cobra.io.load_json_model(eco_directory)
bsub = cobra.io.load_json_model(ijo_directory)
uni = cobra.io.load_json_model(uni_directory)

bsub.optimize()

<Solution 0.12 at 0x7f3e41e8b198>

In [3]:
with open('../me_models/solution.pickle', 'rb') as solution:
    me = pickle.load(solution)

## Essentiality

M-model

In [4]:
from ecolime.analysis import essentiality as es

In [7]:
genes = [gene.id for gene in bsub.genes]
essentiality_df_m,flux_responses_m = es.gene_essentiality(bsub,genes, 'm',NP=10,threshold = 0.01,precision=1e-6)

10 threads: 100%|█████████▉| 843/844 [01:42<00:00,  7.97it/s]

Done


In [8]:
flux_responses_m.to_csv('essentiality_raw_results.csv')

ME-model

In [ ]:
genes = [gene.id for gene in me.genes]
essentiality_df_me,flux_responses_me = es.gene_essentiality(me,genes,'me',NP=10,threshold = 0.01,precision=1e-6)



  0%|          | 0/939 [00:00<?, ?it/s]

10 threads:   0%|          | 0/939 [00:00<?, ?it/s]

10 threads:   0%|          | 1/939 [16:32<258:32:01, 992.24s/it]

10 threads:   0%|          | 2/939 [16:33<180:50:54, 694.83s/it]

10 threads:   0%|          | 3/939 [17:13<129:35:08, 498.41s/it]

10 threads:   0%|          | 4/939 [17:29<91:53:52, 353.83s/it] 

10 threads:   1%|          | 5/939 [17:32<64:29:54, 248.60s/it]

10 threads:   1%|          | 6/939 [20:04<56:54:31, 219.58s/it]

10 threads:   1%|          | 7/939 [20:20<41:00:49, 158.42s/it]

10 threads:   1%|          | 8/939 [34:00<92:17:29, 356.87s/it]

10 threads:   1%|          | 9/939 [37:32<80:59:44, 313.53s/it]

10 threads:   1%|          | 10/939 [40:19<69:34:25, 269.61s/it]

10 threads:   1%|          | 11/939 [1:07:03<172:41:23, 669.92s/it]

10 threads:   1%|▏         | 12/939 [1:22:16<191:15:07, 742.73s/it]

10 threads:   1%|▏         | 13/939 [1:23:48<140:50:32, 547.55s/it]

10 threads:   1%|▏         | 14/939 [2:43:

In [ ]:
flux_responses_me.to_csv('essentiality_raw_me_results.csv')

### Save combined results

In [ ]:
essentiality_df = essentiality_df_m.join(essentiality_df_me,lsuffix='_m',rsuffix='_me')
essentiality_df = essentiality_df_m.join(essentiality_df[['response_me','essentiality_me']].replace(np.nan,'e'))

In [ ]:
essentiality_df.to_csv('essentiality_results.csv')

In [ ]:
essentiality_df[essentiality_df.essentiality_m != essentiality_df.essentiality_me]